## Задание 1
Напишите функцию, которая возвращает название валюты (поле ‘Name’) с максимальным значением курса с помощью сервиса https://www.cbr-xml-daily.ru/daily_json.js

**Вариант 1**

In [28]:
from libs.exchange import Rate

def getting_currency_with_max_value():
    currencies_full_dict = Rate('full').exchange_rates()
    for i, (currency, currency_data) in enumerate(currencies_full_dict.items()):
        if i == 0:
            max_value = currency_data['Value']
        else:
            if currency_data['Value'] > max_value:
                max_value = currency_data['Value']
                currency_with_max_value = currency
    print(f"Валюта с максимальным курсом - {currencies_full_dict[currency_with_max_value]['Name']}")  

In [29]:
getting_currency_with_max_value()

Валюта с максимальным курсом - СДР (специальные права заимствования)


**Вариант 2** (Работает дольше, но сортирует все валюты по текущим значениям в порядке убывания)

In [30]:
from libs.exchange import Rate
from operator import itemgetter

def getting_currency_with_max_value():
    currency_ = Rate('full')
    currencies_dict = {currency:currency_.make_format(currency)['Value'] for currency in currency_.exchange_rates()}
    sorted_currencies_dict = sorted(currencies_dict.items(), key=itemgetter(1),reverse=True)
#     print(sorted_currencies_dict)
    currency_with_max_value = sorted_currencies_dict[0][0]
    print(f"Валюта с максимальным курсом - {currency_.make_format(currency_with_max_value)['Name']}")

In [31]:
getting_currency_with_max_value()

Валюта с максимальным курсом - СДР (специальные права заимствования)


**Вариант 3** (Изменённый Вариант 2. Работает быстрее. Cортирует все валюты по текущим значениям в порядке убывания)

In [35]:
from libs.exchange import Rate
from operator import itemgetter

def getting_currency_with_max_value():
    currency_ = Rate('full')
    currencies_full_dict = currency_.exchange_rates()
    currencies_dict = {currency:currencies_full_dict[currency]['Value'] for currency in currencies_full_dict}
    sorted_currencies_dict = sorted(currencies_dict.items(), key=itemgetter(1),reverse=True)
#     print(sorted_currencies_dict)
    currency_with_max_value = sorted_currencies_dict[0][0]
    print(f"Валюта с максимальным курсом - {currencies_full_dict[currency_with_max_value]['Name']}")

In [36]:
getting_currency_with_max_value()

Валюта с максимальным курсом - СДР (специальные права заимствования)


## Задание 2
Добавьте в класс Rate параметр diff (со значениями True или False), который в случае значения True в методах eur и usd будет возвращать не курс валюты, а изменение по сравнению в прошлым значением. Считайте, что self.diff будет принимать значение True только при возврате значения курса. При отображении всей информации о валюте он не используется.

In [78]:
import requests

class Rate:
    def __init__(self, format_='value'):
        self.format = format_
        if self.format == 'value':
            self.diff = True
        else:
            self.diff = False
        
    def exchange_rates(self):
        """
        Возвращает ответ сервиса с информацией о валютах в виде:
        
        {
            'AMD': {
                'CharCode': 'AMD',
                'ID': 'R01060',
                'Name': 'Армянских драмов',
                'Nominal': 100,
                'NumCode': '051',
                'Previous': 14.103,
                'Value': 14.0879
                },
            ...
        }
        """
        r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
        return r.json()['Valute']
    
    def make_format(self, currency):
        """
        Возвращает информацию о валюте currency в двух вариантах:
        - полная информация о валюте при self.format = 'full':
        Rate('full').make_format('EUR')
        {
            'CharCode': 'EUR',
            'ID': 'R01239',
            'Name': 'Евро',
            'Nominal': 1,
            'NumCode': '978',
            'Previous': 79.6765,
            'Value': 79.4966
        }
        
        Rate('value').make_format('EUR')
        79.4966
        """
        response = self.exchange_rates()
        
        if currency in response:
            if self.format == 'full':
                return response[currency]
            
            if self.format == 'value':
                return response[currency]['Value']
        
        return 'Error'
    
    def eur(self):
        """
        Возвращает полную информацию о валюте в формате self.format = 'full' или 
        разницу между текущим и предыдущим значениями курса, если self.format = 'value'
        
        """
        self.format = 'full'
        eur_info = self.make_format('EUR')
        if self.diff == True:
            eur_diff = eur_info['Value'] - eur_info['Previous']
            self.format = 'value'
            return eur_diff
        return eur_info       
    
    def usd(self):
        """
        Возвращает полную информацию о валюте в формате self.format = 'full' или 
        разницу между текущим и предыдущим значениями курса, если self.format = 'value'
        """
        self.format = 'full'
        usd_info = self.make_format('USD')
        if self.diff == True:
            usd_diff = usd_info['Value'] - usd_info['Previous']
            self.format = 'value'
            return usd_diff
        return usd_info

    def AZN(self):
        """Возвращает курс азербайджанского маната на сегодня в формате self.format"""
        return self.make_format('AZN')

In [81]:
r = Rate(format_='full')
print(r.eur()) #Возвращает полную информацию о валюте
print(r.usd()) #Возвращает полную информацию о валюте
print(r.AZN()) #Возвращает полную информацию о валюте

{'ID': 'R01239', 'NumCode': '978', 'CharCode': 'EUR', 'Nominal': 1, 'Name': 'Евро', 'Value': 80.2528, 'Previous': 81.1468}
{'ID': 'R01235', 'NumCode': '840', 'CharCode': 'USD', 'Nominal': 1, 'Name': 'Доллар США', 'Value': 74.7163, 'Previous': 75.129}
{'ID': 'R01020A', 'NumCode': '944', 'CharCode': 'AZN', 'Nominal': 1, 'Name': 'Азербайджанский манат', 'Value': 44.0414, 'Previous': 44.2847}


In [82]:
r = Rate()
print(r.eur()) #Возвращает разницу текущего и предыдущего курсов
print(r.usd()) #Возвращает разницу текущего и предыдущего курсов
print(r.AZN()) #Возвращает текущий курс

-0.8940000000000055
-0.41270000000000095
44.0414


## Задание 3
Напишите класс Designer, который учитывает количество международных премий для дизайнеров (из презентации: "Повышение на 1 грейд за каждые 7 баллов. Получение международной премии – это +2 балла"). Считайте, что при выходе на работу сотрудник уже имеет две премии и их количество не меняется со стажем (конечно если хотите это можно вручную менять). Выполните проверку для 20 аккредитаций дизайнера Елены.

In [8]:
class Employee:
    def __init__(self, name, seniority):
        self.name = name
        self.seniority = seniority  
        self.grade = 1
    
    def grade_up(self):
        """Повышает уровень сотрудника"""
        print(f'{self.name}, поздравляем! Вы повышены!')
        self.grade += 1
    
    def publish_grade(self):
        """Публикация результатов аккредитации сотрудников"""
        print(self.name, self.grade)

In [9]:
class Designer(Employee):
    def __init__(self, name, seniority, awards=2):
        super().__init__(name, seniority=2*awards)
    
    def check_if_it_is_time_for_upgrade(self):
        self.seniority += 1
        if self.seniority % 7 == 0:
            self.grade_up()
        return self.publish_grade()

In [10]:
elena = Designer('Елена', seniority=0, awards=2)

In [11]:
for i in range(20):
    elena.check_if_it_is_time_for_upgrade()

Елена 1
Елена 1
Елена, поздравляем! Вы повышены!
Елена 2
Елена 2
Елена 2
Елена 2
Елена 2
Елена 2
Елена 2
Елена, поздравляем! Вы повышены!
Елена 3
Елена 3
Елена 3
Елена 3
Елена 3
Елена 3
Елена 3
Елена, поздравляем! Вы повышены!
Елена 4
Елена 4
Елена 4
Елена 4
